In [1]:
import pandas as pd 
import numpy as np 

In [2]:
df = pd.read_csv('quikr_car.csv')

In [3]:
df['kms_driven']

0      45,000 kms
1          40 kms
2      22,000 kms
3      28,000 kms
4      36,000 kms
          ...    
887           NaN
888    27,000 kms
889    40,000 kms
890        Petrol
891        Petrol
Name: kms_driven, Length: 892, dtype: object

In [4]:
#Data cleaning 
#1) year has many non year values : str.isnumeric
#2) convert year object is in INT : astype(int)
#3) price has 'ask for price' : 
#4) convert price to int :
#5) remove KMS from kms_driven : .str.split() & str.get() & str.replace(',' , '') & str.isnumeric
#6) convert from obj to int 
#7) has nan values 
#8) nan values in fuel type 
#9) keep only first 3 words in name 

In [5]:
df = df[df['year'].str.isnumeric()]

In [6]:
df['year'] = df['year'].astype(int)

In [7]:
df = df[df['Price'] != 'Ask For Price']

In [8]:
df['Price'] = df['Price'].apply(lambda x : x.replace(',' , ''))

In [9]:
df['Price'] = df['Price'].astype(int)

In [10]:
df['kms_driven'] = df['kms_driven'].str.split().str.get(0).str.replace(',' , '')

In [11]:
df = df[df['kms_driven'].str.isnumeric()] 

In [12]:
df['kms_driven'] = df['kms_driven'].astype(int)

In [13]:
df['fuel_type'].isna().sum()

1

In [14]:
df.dropna(subset= 'fuel_type' ,inplace=True)

In [15]:
df.isna().sum()

name          0
company       0
year          0
Price         0
kms_driven    0
fuel_type     0
dtype: int64

In [16]:
df['name'] = df['name'].str.split(' ').str.slice(0,3).str.join(' ')

In [17]:
#reset index , #the second step is not usually needed
df.reset_index(drop=True)

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing,Hyundai,2007,80000,45000,Petrol
1,Mahindra Jeep CL550,Mahindra,2006,425000,40,Diesel
2,Hyundai Grand i10,Hyundai,2014,325000,28000,Petrol
3,Ford EcoSport Titanium,Ford,2014,575000,36000,Diesel
4,Ford Figo,Ford,2012,175000,41000,Diesel
...,...,...,...,...,...,...
811,Maruti Suzuki Ritz,Maruti,2011,270000,50000,Petrol
812,Tata Indica V2,Tata,2009,110000,30000,Diesel
813,Toyota Corolla Altis,Toyota,2009,300000,132000,Petrol
814,Tata Zest XM,Tata,2018,260000,27000,Diesel


In [18]:
#outliers check 
df = df[df['Price'] < 6000000].reset_index(drop=True)

In [19]:
df

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing,Hyundai,2007,80000,45000,Petrol
1,Mahindra Jeep CL550,Mahindra,2006,425000,40,Diesel
2,Hyundai Grand i10,Hyundai,2014,325000,28000,Petrol
3,Ford EcoSport Titanium,Ford,2014,575000,36000,Diesel
4,Ford Figo,Ford,2012,175000,41000,Diesel
...,...,...,...,...,...,...
810,Maruti Suzuki Ritz,Maruti,2011,270000,50000,Petrol
811,Tata Indica V2,Tata,2009,110000,30000,Diesel
812,Toyota Corolla Altis,Toyota,2009,300000,132000,Petrol
813,Tata Zest XM,Tata,2018,260000,27000,Diesel


In [20]:
x = df.drop(columns='Price')
y = df['Price']

In [21]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.2)

In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline 

In [23]:
ohe = OneHotEncoder()
ohe.fit(x[['name' , 'company' , 'fuel_type']])
ohe.categories_

[array(['Audi A3 Cabriolet', 'Audi A4 1.8', 'Audi A4 2.0', 'Audi A6 2.0',
        'Audi A8', 'Audi Q3 2.0', 'Audi Q5 2.0', 'Audi Q7', 'BMW 3 Series',
        'BMW 5 Series', 'BMW 7 Series', 'BMW X1', 'BMW X1 sDrive20d',
        'BMW X1 xDrive20d', 'Chevrolet Beat', 'Chevrolet Beat Diesel',
        'Chevrolet Beat LS', 'Chevrolet Beat LT', 'Chevrolet Beat PS',
        'Chevrolet Cruze LTZ', 'Chevrolet Enjoy', 'Chevrolet Enjoy 1.4',
        'Chevrolet Sail 1.2', 'Chevrolet Sail UVA', 'Chevrolet Spark',
        'Chevrolet Spark 1.0', 'Chevrolet Spark LS', 'Chevrolet Spark LT',
        'Chevrolet Tavera LS', 'Chevrolet Tavera Neo', 'Datsun GO T',
        'Datsun Go Plus', 'Datsun Redi GO', 'Fiat Linea Emotion',
        'Fiat Petra ELX', 'Fiat Punto Emotion', 'Force Motors Force',
        'Force Motors One', 'Ford EcoSport', 'Ford EcoSport Ambiente',
        'Ford EcoSport Titanium', 'Ford EcoSport Trend',
        'Ford Endeavor 4x4', 'Ford Fiesta', 'Ford Fiesta SXi', 'Ford Figo',
        '

In [26]:
transformer = ColumnTransformer(transformers=[
    ('trans1', OneHotEncoder(categories=ohe.categories_) , ['name' , 'company' , 'fuel_type'])] , remainder= 'passthrough')

In [27]:
lr = LinearRegression()

In [28]:
pipe = make_pipeline(transformer , lr)

In [31]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('trans1',
                                                  OneHotEncoder(categories=[array(['Audi A3 Cabriolet', 'Audi A4 1.8', 'Audi A4 2.0', 'Audi A6 2.0',
       'Audi A8', 'Audi Q3 2.0', 'Audi Q5 2.0', 'Audi Q7', 'BMW 3 Series',
       'BMW 5 Series', 'BMW 7 Series', 'BMW X1', 'BMW X1 sDrive20d',
       'BMW X1 xDrive20d', 'Chevrolet Beat', 'Chevrolet Beat Diesel'...
                                                                            array(['Audi', 'BMW', 'Chevrolet', 'Datsun', 'Fiat', 'Force', 'Ford',
       'Hindustan', 'Honda', 'Hyundai', 'Jaguar', 'Jeep', 'Land',
       'Mahindra', 'Maruti', 'Mercedes', 'Mini', 'Mitsubishi', 'Nissan',
       'Renault', 'Skoda', 'Tata', 'Toyota', 'Volkswagen', 'Volvo'],
      dtype=object),
                                                                            array(['Diesel', 'LPG', 'P

In [32]:
y_pred = pipe.predict(x_test)
y_pred 

array([ 3.29540967e+05,  4.69356410e+05,  3.19544107e+05,  2.28686420e+05,
        3.42539522e+05,  3.29540967e+05,  2.20747393e+05,  3.16776191e+05,
        2.25968816e+05,  2.37181050e+05,  8.39074352e+05,  4.87038545e+05,
        3.63662951e+05,  2.39035181e+05,  5.38277967e+05,  6.90866149e+04,
       -1.08130740e+05,  1.16341392e+05,  4.52253622e+05, -6.83809874e+04,
        1.17213391e+04,  4.72401983e+05,  5.74113955e+05,  1.83682905e+05,
        5.28831703e+05,  3.69603959e+05,  8.90234767e+04,  2.23924335e+05,
        8.56320778e+05,  6.95743900e+04,  3.14415397e+05,  4.89699269e+05,
        2.08303847e+05,  2.44696687e+05,  3.14416474e+05,  2.68665676e+05,
        3.84087624e+05,  3.99500942e+05,  3.18222256e+05,  1.75101064e+05,
        3.84369924e+05,  5.29569849e+05,  2.29897704e+05,  3.16189265e+05,
        3.41141422e+05,  2.63743185e+05,  2.10455917e+05,  3.78393069e+05,
        1.93336025e+05,  5.12609053e+05,  2.49807879e+05,  3.94282685e+05,
        2.88024982e+05,  

In [33]:
r2_score(y_pred,y_test)

0.6593466754320263

In [36]:
scores = []
for i in range(1000):
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2 , random_state=i)
    lr = LinearRegression()
    pipe = make_pipeline(transformer , lr)
    pipe.fit(x_train,y_train)
    y_pred = pipe.predict(x_test)
    scores.append(r2_score(y_pred,y_test))

In [39]:
scores[np.argmax(scores)]

0.8778387651679336

In [41]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2 , random_state=124)
lr = LinearRegression()
pipe = make_pipeline(transformer , lr)
pipe.fit(x_train,y_train)
y_pred = pipe.predict(x_test)
r2_score(y_pred,y_test)

0.8778387651679336

In [42]:
import pickle 

In [45]:
pickle.dump(pipe,open('LinearRegressionModel.pkl','wb'))

In [46]:
df.to_csv('clean car.csv')